In [1]:
import sys
#!{sys.executable} -m pip install plotly 
# !{sys.executable} -m pip install sae_lens transformer_lens
import sys
# !{sys.executable} -m pip install plotly torch huggingface_hub sae_lens transformer_lens sae_dashboard
!{sys.executable} -m pip install torch huggingface_hub sae_lens transformer_lens sae_dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 15.2 MB/s eta 0:00:00
  Using cached typer-0.12.5-py3-none-any.whl (47 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 KB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 KB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 34.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 KB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
try:
    import google.colab # type: ignore
    from google.colab import output
    COLAB = True
    # %pip install sae-lens transformer-lens sae-dashboard
except:
    COLAB = False
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# Standard imports
import os
import torch
from tqdm import tqdm
import plotly.express as px
import pandas as pd

# Imports for displaying vis in Colab / notebook

torch.set_grad_enabled(False)

# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

device


'cuda'

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
from sae_lens import SAE, HookedSAETransformer

MODEL_ID = "gemma-2-9b-it"
LAYER_NAME = "blocks.31.hook_resid_post"
NEURONPEDIA_SAE_NAME = "31-gemmascope-res-131k"
SAE_RELEASE = "gemma-scope-9b-it-res-canonical"
SAE_ID = "layer_31/width_131k/canonical"

# MODEL_ID = "gemma-2-2b-it"
# LAYER_NAME = "blocks.20.hook_resid_post"
# NEURONPEDIA_SAE_NAME = "20-axbench-reft-r1-res-16k"
# SAE_RELEASE = "pyvene/gemma-reft-r1-2b-it-res"
# SAE_ID = "l20"

model = HookedSAETransformer.from_pretrained(MODEL_ID, device=device)


# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
torch.cuda.empty_cache()
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = SAE_RELEASE, # see other options in sae_lens/pretrained_saes.yaml
    # sae_id = "layer_20/width_16k/average_l0_71", # won't always be a hook point
    # sae_id = "layer_20/width_65k/average_l0_114",
    sae_id = SAE_ID,
    device = device
)



config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2-9b-it into HookedTransformer


params.npz:   0%|          | 0.00/3.76G [00:00<?, ?B/s]

In [14]:
# Function to generate text
def generate_text(prompt, max_new_tokens=300, temperature=0.7):
    tokens = model.to_tokens(prompt)
    output = model.generate(
        tokens, 
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
    )
    return model.tokenizer.decode(output[0])

# Ask about Greg Dean's joke mine method
prompt = "Explain Greg Dean's joke mine method for creating jokes:"
generated_text = generate_text(prompt)

print(generated_text)

  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Explain Greg Dean's joke mine method for creating jokes:

Greg Dean's joke mine method is a structured approach to joke writing that involves digging deep into personal experiences and observations to unearth comedic gold. Here's a breakdown of the method:

**1. The Mine Shaft:** This refers to your own life experiences, memories, and even everyday observations. It's the source material for your jokes. Dean emphasizes the importance of actively paying attention to the world around you and noting the funny or absurd things you encounter.

**2. The Pickaxe:** This is your critical thinking and analytical skills. You need to sift through your experiences and identify the nuggets of humor within them. Ask yourself: What's unexpected? What's ironic? What's relatable?

**3. The Gold:** The "gold" is the actual joke itself. It's the punchline, the clever twist, or the unexpected observation that makes people laugh. Dean believes that the best jokes come from a place of truth and authenti

In [15]:
print(generate_text("Analyze why this joke is funny: 'When life gives you melons, you might be dyslexic'"))



  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny: 'When life gives you melons, you might be dyslexic'

**Why it's funny:**

This joke relies on a **play on words** and a **misunderstanding**.

* **Wordplay:** The joke uses the similar-sounding words "melons" and "problems."  This creates a humorous confusion because we expect the phrase "when life gives you lemons" to mean facing difficulties.
* **Dyslexia:** The punchline introduces dyslexia, a learning disorder that affects reading and spelling. By suggesting that someone might be dyslexic because they perceive "melons" instead of "lemons," the joke playfully highlights the potential for misinterpretation caused by dyslexia.

**The humor arises from:**

* **Unexpected Twist:** The joke subverts our expectations by replacing the common idiom with a nonsensical alternative.
* **Relatability:** Many people are familiar with the idiom "when life gives you lemons, make lemonade," so the unexpected change creates a humorous jolt.
* **Lightheartedness:*

In [16]:
sentences = {
    "original": "When life gives you lemons, make lemonade",
    "melons": "When life gives you melons, you might be dyslexic",
    "aids": "When life gives you AIDS, make lemonaids",
    "take back": "When life gives you lemons, don't make lemonade. Make life take the lemons back! Get mad! I don't want your damn lemons! What the hell am I supposed to do with these? Demand to see life's manager! Make life rue the day it thought it could give Cave Johnson lemons! Do you know who I am? I'm the man who's gonna burn your house down! With the lemons! I'm gonna get my engineers to invent a combustible lemon that burns your house down!",
    "bail": "When life gives you lemons, just say 'fuck the lemons' and bail.",
    "resell": "When life gives you lemons, resell them to a grocer for as high of a profit margin as you can",
    "fuck the lemons": "When life gives you lemons, say 'fuck the lemons' and squeeze them in life's eyes."
}

for joke in list(sentences.values())[:]:
    print(generate_text(f"Analyze why this joke is funny:\n\n{joke}\n\n"))

  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

When life gives you lemons, make lemonade

**Why it's funny:**

* **Unexpected twist:** The joke sets up a scenario where life presents lemons, which are typically seen as unpleasant. However, the punchline suggests a positive and resourceful solution – making lemonade. This unexpected shift in tone creates humor.
* **Subversion of expectations:** We expect life to throw us challenges and hardships, but the joke playfully subverts this expectation by offering a simple and optimistic solution.
* **Irony:** There's a subtle irony in the fact that making lemonade from lemons is a cliché. The joke acknowledges this familiarity while still finding humor in its simplicity and unexpectedness.
* **Relatability:**  Most people can relate to the feeling of facing difficulties in life. The joke's message of turning lemons into lemonade resonates with this shared experience, making it more humorous.


Let me know if you'd like to explore the humor of other jok

  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

When life gives you melons, you might be dyslexic

**Explanation:**

This joke relies on a **play on words** and a **misunderstanding**. 

* **The Setup:** "When life gives you melons..." sets up the expectation of a common proverb, "When life gives you lemons, make lemonade." This proverb suggests resilience and turning adversity into something positive.

* **The Punchline:** "...you might be dyslexic" subverts this expectation. Dyslexia is a learning disorder that affects reading and spelling. The joke implies that because "melons" sounds like "lemons," someone with dyslexia might misinterpret the proverb.

**Why it's funny:**

* **Unexpected Twist:** The punchline is unexpected and surprising, breaking the pattern established by the setup.
* **Wordplay:** The joke cleverly uses the similar sounds of "melons" and "lemons" to create humor.
* **Relatability:** Many people know the proverb and can easily understand the context, making the unexpected

  0%|          | 0/300 [00:00<?, ?it/s]

...

<bos>Analyze why this joke is funny:

When life gives you lemons, just say 'fuck the lemons' and bail.

**Explanation:**

This joke is funny because it subverts expectations and offers a cynical, humorous take on a common piece of advice. 

Here's a breakdown:

* **The Setup:** "When life gives you lemons..." is a well-known proverb encouraging resilience and making the best of a bad situation. We expect a response that aligns with this optimistic sentiment, like "make lemonade."

* **The Twist:**  The punchline, "just say 'fuck the lemons' and bail," completely flips the script. Instead of embracing the challenge, it suggests a defiant and cowardly response.

* **Humor in Subversion:** The humor comes from the unexpectedness and absurdity of the twist. It's funny because it's so contrary to the expected advice, highlighting the absurdity of forcing positivity in all situations.

* **Relatability:**  Many people can relate to the feeling of wanting to just give up when faced with diffi

  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

When life gives you lemons, resell them to a grocer for as high of a profit margin as you can

**Explanation:**

This joke plays on the common idiom "When life gives you lemons, make lemonade." 

Here's why it's funny:

* **Subversion of Expectations:** We expect a response that focuses on making something positive out of a difficult situation (like lemonade). Instead, the joke takes a cynical and capitalist approach, suggesting that the best course of action is to exploit the situation for personal gain.
* **Unexpectedly Realistic:**  While making lemonade is a wholesome and optimistic solution, the joke's suggestion of reselling lemons highlights a more pragmatic (and perhaps cynical) perspective.  It acknowledges that sometimes the most "successful" response isn't always the most ethical or heartwarming.
* **Wordplay:** The phrase "high profit margin" adds a layer of humor by emphasizing the absurdity of treating lemons like a commodity to be ma

  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

When life gives you lemons, say 'fuck the lemons' and squeeze them in life's eyes.

**Why it's funny:**

* **Subversion of Expectations:** The joke plays on the common idiom "When life gives you lemons, make lemonade." This idiom encourages optimism and resourcefulness in the face of adversity. By saying "fuck the lemons" and squeezing them in life's eyes, the joke completely flips this expectation, offering a darkly humorous and aggressive response.

* **Absurdity:** The image of squeezing lemons into someone's eyes is inherently absurd and unexpected. This unexpected twist adds to the humor, as it's a ridiculous and illogical action.

* **Dark Humor:** The joke has a dark and cynical edge to it.  Instead of accepting lemons as a challenge, the punchline suggests a rebellious and even vengeful reaction. This dark humor appeals to those who find humor in the unconventional and the slightly unsettling.

* **Cathartic Release:** For some, the joke mi

In [17]:
def get_last_token_features(sentence):
    # Tokenize the sentence
    tokens = model.to_tokens(sentence)
    
    # Store activations in this dict
    activation_dict = {}
    
    # Function to capture activations only (don't modify the forward pass)
    def hook_fn(act, hook):
        activation_dict['act'] = act.detach().clone()
        return act  # Return unchanged activations
    
    # Run the model with the hook
    layer_name = LAYER_NAME  # Use the layer matching your sae_id
    with model.hooks([(layer_name, hook_fn)]):
        model(tokens)
    
    # Now encode the captured activations with your SAE
    acts = activation_dict['act']
    with torch.no_grad():
        # Reshape to 2D for SAE encoding
        batch_size, seq_len, hidden_dim = acts.shape
        reshaped_acts = acts.reshape(-1, hidden_dim)
        
        # Encode with SAE
        feature_acts = sae.encode(reshaped_acts)
        
        # Reshape back to match original dimensions but with SAE feature dimension
        feature_acts = feature_acts.reshape(batch_size, seq_len, -1)
    
    # Calculate mean activations across tokens
    # mean_acts = feature_acts.mean(dim=1).squeeze(0)
    last_token_acts = feature_acts[0, -1, :]  # [batch=0, last_position, all_features]

    return last_token_acts

In [59]:
import requests
NEURONPEDIA_API_KEY = "d35ad1b5-cbb0-454c-8ed2-86fe9d221c55"

def get_feature_description(json_data):
    """Extract the feature description from Neuronpedia SAE JSON data."""
    
    # Look for the explanation with the highest score or first available description
    for explanation in json_data.get("explanations", []):
        if "description" in explanation:
            return explanation["description"].strip()
    
    return "No description available"

def get_neuronpedia_feature(feature_idx, model_id=MODEL_ID, layer=NEURONPEDIA_SAE_NAME):
    url = f"https://www.neuronpedia.org/api/feature/{model_id}/{layer}/{feature_idx}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    response.raise_for_status()
 

# Get feature activations for each sentence
feature_activations = []

for sentence in sentences.values():
    last_token_acts = get_last_token_features(sentence + 3 * " ")
    feature_activations.append(last_token_acts)

# Find the differences between control and each sentence
differences = []
for i in range(len(sentences)):
    diff = feature_activations[i] - feature_activations[0]  # Using control sentence as baseline
    differences.append(diff)

# Convert differences to a tensor where each row represents a sentence
diff_tensor = torch.stack(differences)  # Shape: [num_sentences, num_features]

# Get the features that show the most consistent differences across jokes
# Take the absolute mean difference across all sentences
# mean_abs_diff = torch.abs(diff_tensor).mean(dim=0)
mean_abs_diff = diff_tensor.mean(dim=0)  # Average absolute change per feature


# Find the top features by mean absolute difference
top_feature_indices = torch.topk(mean_abs_diff, k=50).indices

# Create a summary of these top features across all jokes
print("Top 50 joke-related features (most consistently different from control):")
print(f"{'Feature':<10} {'Mean Abs Diff':<15} {'Mean Diff':<15}")
print("-" * 50)

for idx in top_feature_indices:
    idx_val = idx.item()
    mean_abs = mean_abs_diff[idx_val].item()
    mean_diff = diff_tensor[:, idx_val].mean().item()  # Actual mean with sign
    
    feature_info = get_neuronpedia_feature(idx_val)
    
    description = get_feature_description(feature_info)
    
    # Print stats with description
    print(f"{idx_val:<10} {mean_diff:.6f}        {description}")
    
    # Print per-sentence activation differences for this feature
    print("Per joke differences:")
    for j, (shorthand, sentence) in enumerate(sentences.items()):
        if j == 0:
            print(f"  {feature_activations[0][idx_val].item():.6f}")

        if j > 0: # skip original
            print(f"  {shorthand.ljust(20)}: {diff_tensor[j, idx_val].item():.6f}")
    print()

print("\n\Original proverb activations")
for idx in top_feature_indices:
    idx_val = idx.item()


Top 50 joke-related features (most consistently different from control):
Feature    Mean Abs Diff   Mean Diff      
--------------------------------------------------
43409      20.179413        mentions of Reddit and discussions related to its functionality
Per joke differences:
  melons              : 11.508745
  aids                : 12.706074
  take back           : 35.192154
  bail                : 30.833200
  resell              : 25.985165
  fuck the lemons     : 25.030556

100823     18.641743        expressions of humor, especially in dialogues and narratives
Per joke differences:
  melons              : 44.336456
  aids                : 23.920622
  take back           : 9.621561
  bail                : 21.431643
  resell              : 13.317831
  fuck the lemons     : 17.864069

1238       14.492020        expressions of excitement or enthusiasm
Per joke differences:
  melons              : 9.801444
  aids                : 11.934306
  take back           : 11.703902
  bail  

In [50]:
import requests

def search_neuronpedia(query, model_id=MODEL_ID, layer=NEURONPEDIA_SAE_NAME):
    url = "https://www.neuronpedia.org/api/explanation/search"
    
    payload = {
        "modelId": MODEL_ID,
        "layers": [layer],
        "query": query,
    }
    headers = {"Content-Type": "application/json"}

    has_more = True
    result = []
    
    while has_more:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        j = response.json()
        new_results = [(int(r['index']), r['description'].strip()) for r in j["results"]]
        if not any(query in r[1] for r in new_results):
            print("No query in any of the", len(new_results), "results")
            break
        result.extend(new_results)
        payload["offset"] = j['nextOffset']
        has_more = j['hasMore']
        print(j['nextOffset'], j['hasMore'])
    
    return result


def find_jokes_features_on_neuronpedia(model_id=MODEL_ID, layer=NEURONPEDIA_SAE_NAME):
    results1 = search_neuronpedia("humor", model_id, layer)
    results2 = search_neuronpedia("joke", model_id, layer)
    results3 = search_neuronpedia("comedy", model_id, layer)


    return {**dict(results1), **dict(results2), **dict(results3)}

joke_features = find_jokes_features_on_neuronpedia()

20 True
40 True
60 True
80 True
100 True
No query in any of the 20 results
20 True
No query in any of the 20 results
20 True
No query in any of the 20 results


In [51]:
def stricter_filtered_joke_features(joke_features):
    return dict((k,v) for (k,v) in joke_features.items() if 'joke' in v or 'humor' in v or 'comedy' in v) 

print(len(joke_features), len(stricter_filtered_joke_features(joke_features)))
fires = 0

for idx in stricter_filtered_joke_features(joke_features):
    idx_val = idx
    mean_abs = mean_abs_diff[idx_val].item()
    mean_diff = diff_tensor[:, idx_val].mean().item()  # Actual mean with sign
    
    # feature_info = get_neuronpedia_feature(idx_val)
    # description = get_feature_description(feature_info)
    description = joke_features[idx]
    
    # Print stats with description
    print(f"{idx_val:<10} {mean_diff:.6f}        {description}")
    
    # Print per-sentence activation differences for this feature
    if not (diff_tensor[:, idx_val] == 0).all().item():
        fires += 1
        for j, (shorthand, sentence) in enumerate(sentences.items()):
            print(f"{shorthand.ljust(17)} {diff_tensor[j, idx_val].item():.6f}")
    print()

print(fires, len(stricter_filtered_joke_features(joke_features)))

100 49
96961      0.000000        expressions of humor and comedy

70032      0.000000        words related to "humor" and its various forms

100823     18.641743        expressions of humor, especially in dialogues and narratives
original          0.000000
melons            44.336456
aids              23.920622
take back         9.621561
bail              21.431643
resell            13.317831
fuck the lemons   17.864069

73520      2.299195        elements of humor and laughter within the text
original          0.000000
melons            16.094362
aids              0.000000
take back         0.000000
bail              0.000000
resell            0.000000
fuck the lemons   0.000000

38789      0.000000        expressions of humor and laughter

107128     0.000000        references to humor or jokes

2730       -1.778948        references to humor or jokes
original          0.000000
melons            13.049977
aids              6.772878
take back         -8.068873
bail              -8.06

In [21]:
for idx in joke_features:
    description = joke_features[idx]
    
    # Print stats with description
    print(f"{idx:<10}  {description}")
    
    # Print per-sentence activation differences for this feature
    if not all(f[idx] == 0.0 for f in feature_activations):
        for j, (shorthand, sentence) in enumerate(sentences.items()):
            print(f"{shorthand.ljust(17)} {feature_activations[j][idx].item():.6f}")
    print()

96961       expressions of humor and comedy

70032       words related to "humor" and its various forms

100823      expressions of humor, especially in dialogues and narratives
original          0.000000
melons            0.000000
aids              0.000000
take back         0.000000
bail              9.354599
resell            0.000000
fuck the lemons   0.000000

73520       elements of humor and laughter within the text

38789       expressions of humor and laughter

107128      references to humor or jokes

2730        references to humor or jokes

69623       references to humor and practical jokes
original          0.000000
melons            0.000000
aids              0.000000
take back         12.093092
bail              0.000000
resell            0.000000
fuck the lemons   0.000000

3835        instances of the word "humor" and related variations in various contexts

9577        references to amusement and humor

3513        expressions related to humor and amusement
original  

In [22]:
# https://parade.com/1024249/marynliles/funny-puns/
puns = [
    "To the guy who invented zero, thanks for nothing.", 
    "What did the grape say when it got crushed? Nothing, it just let out a little wine.",
    "Time flies like an arrow. Fruit flies like a banana.",
    "Ladies, if he can’t appreciate your fruit jokes, you need to let that mango.",
    "A man sued an airline company after it lost his luggage. Sadly, he lost his case.",
    "I was wondering why the ball was getting bigger. Then it hit me.",
    "Never trust an atom, they make up everything!",
    "Waking up this morning was an eye-opening experience",
    "Getting the ability to fly would be so uplifting.",
    "England doesn't have a kidney bank, but it does have a Liverpool.",
    "German sausage jokes are just the wurst.",
    "I bought a boat because it was for sail.",
    "A cross-eyed teacher couldn’t control his pupils.",
    "The machine at the coin factory just suddenly stopped working, with no explanation. It doesn’t make any cents.",
    "My ex-wife still misses me. But her aim is starting to improve!",
    "Why are frogs so happy? They eat whatever bugs them",
    "So what if I don’t know what apocalypse means? It’s not the end of the world!",
    "She had a photographic memory, but never developed it.",
    "I'm reading a book about anti-gravity. It's impossible to put down!",
    "I had a crazy dream last night! I was swimming in an ocean of orange soda. Turns out it was just a Fanta sea."
]

non_puns = [
    "To the guy who invented zero, thank you.", 
    "What did the grape say when it got crushed? Nothing, it couldn't speak",
    "Time flies fast.",
    "Ladies, if he can’t appreciate your jokes, you need to leave him.",
    "A man sued an airline company after it lost his luggage. He won.",
    "I was wondering why the ball hit me.",
    "Atoms make up everything.",
    "Waking up this morning was great.",
    "Getting the ability to fly would be so great.",
    "England does have a kidney bank.",
    "German sausages are great.",
    "I bought a boat because it was for sale.",
    "A cross-eyed teacher had some vision problems.",
    "The machine at the factory just suddenly stopped working, with no explanation.",
    "My ex-wife still misses me.",
    "Why are frogs so happy? They are frogs.",
    "So what if I don’t know what apocalypse means? It’s one word.",
    "She had a photographic memory.",
    "I'm reading a book about anti-gravity.",
    "I had a crazy dream last night! I was swimming in an ocean of soda."
]

In [23]:
for pun in puns[:7]:
    print(generate_text(f"Analyze why this joke is funny:\n\n{pun}.\n Here's why:\n\n"))

  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

To the guy who invented zero, thanks for nothing..
 Here's why:

* **Wordplay:** The joke relies on the double meaning of "thanks for nothing."  
    * **Literal meaning:** The speaker is expressing gratitude, but sarcastically, as zero is considered worthless. 
    * **Figurative meaning:**  The joke plays on the idea that zero represents the absence of something.

* **Unexpected Twist:** The setup leads us to expect a sincere thank you, but the punchline subverts our expectations with a sarcastic twist.
* **Absurdity:** The idea of thanking someone for inventing something seemingly useless is inherently absurd, which adds to the humor.


Let me know if you'd like to explore the humor in other jokes!<end_of_turn>
<eos>


  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

What did the grape say when it got crushed? Nothing, it just let out a little wine..
 Here's why:

* **Wordplay:** The joke relies on the double meaning of "let out."  We usually associate "let out" with a sound, like a scream or a laugh. However, in this case, it refers to the act of grapes being pressed to extract wine. 

* **Unexpected Twist:** The setup leads you to expect a verbal response from the grape.  The punchline subverts this expectation by using "wine" as a play on words, creating a humorous surprise.

* **Relatability:**  Most people are familiar with the process of making wine and the concept of grapes being crushed. This shared knowledge makes the joke more accessible and understandable.



Let me know if you'd like to explore other jokes and their comedic elements!
<end_of_turn><eos>


  0%|          | 0/300 [00:00<?, ?it/s]

...

<bos>Analyze why this joke is funny:

A man sued an airline company after it lost his luggage. Sadly, he lost his case..
 Here's why:

* **Unexpected Twist:** The joke sets up an expectation of a typical lawsuit scenario where the man is likely to win compensation. However, the punchline delivers an unexpected twist by stating that he lost his case, creating a humorous dissonance.

* **Wordplay:** The joke cleverly uses the word "lost" in two different contexts. First, it refers to the airline losing the man's luggage, and then it refers to the man losing his lawsuit. This double meaning adds to the humor by creating a playful ambiguity.

* **Absurdity:** The situation is slightly absurd.  It's funny to imagine someone losing a lawsuit because the airline lost their luggage, highlighting the sometimes illogical nature of legal proceedings.



Let me know if you'd like to explore other jokes or humor theories! 
<end_of_turn><eos>


  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

I was wondering why the ball was getting bigger. Then it hit me..
 Here's why:

* **Unexpected Twist:** The setup leads you to believe the joke will be about the physical size of a ball increasing.  The punchline subverts this expectation by using "hit me" in a metaphorical sense, referring to a sudden realization.

* **Wordplay:** The joke relies on the double meaning of "hit me."  It plays on the literal act of being struck by a ball and the figurative meaning of understanding something.

* **Simplicity:** The joke is short, concise, and easy to understand.  This makes it accessible to a wide audience and allows the humor to land quickly.


Let me know if you'd like to explore the humor in other jokes! 
<end_of_turn><eos>


  0%|          | 0/300 [00:00<?, ?it/s]

<bos>Analyze why this joke is funny:

Never trust an atom, they make up everything!.
 Here's why:

* **Wordplay:** The joke relies on the double meaning of "make up."  

    *  It can mean "to invent" or "to lie."  
    * It also means "to compose" or "to constitute."

* **Unexpected Twist:** The setup leads you to expect a serious or negative statement about atoms.  The punchline subverts this expectation with a playful, lighthearted observation about their fundamental role in the universe.
* **Scientific Basis:** The joke has a basis in scientific fact. Atoms are indeed the building blocks of all matter. This grounding in reality adds to the humor by highlighting the absurdity of the situation.

Let me know if you'd like to explore the humor in other jokes! 


<end_of_turn><eos>


In [56]:
pun_acts = []
for pun in puns:
    pun_acts.append((get_last_token_features(pun), get_last_token_features(pun + 3 * " ")))

non_pun_acts = []
for non_pun in non_puns:
    non_pun_acts.append((get_last_token_features(non_pun), get_last_token_features(non_pun + 3 * " ")))


for idx in joke_features:
    description = joke_features[idx]
    
    # Print stats with description
    print(f"{idx:<10}  {description}")
    
    if not all(f[1][idx] == 0.0 and f[0][idx] == 0.0 for f in pun_acts):
        for j, pun in enumerate(puns):
            print(f"   {pun_acts[j][0][idx].item():6.2f} {pun_acts[j][1][idx].item():6.2f}  {pun}")

        print("--- control non-puns ---")
        for j, non_pun in enumerate(non_puns):
            print(f"   {non_pun_acts[j][0][idx].item():6.2f} {non_pun_acts[j][1][idx].item():6.2f}  {non_pun}")
        print()

96961       expressions of humor and comedy
70032       words related to "humor" and its various forms
100823      expressions of humor, especially in dialogues and narratives
    14.81  42.05  To the guy who invented zero, thanks for nothing.
    53.03  35.59  What did the grape say when it got crushed? Nothing, it just let out a little wine.
    22.87  27.55  Time flies like an arrow. Fruit flies like a banana.
     0.00  35.44  Ladies, if he can’t appreciate your fruit jokes, you need to let that mango.
    12.73   9.81  A man sued an airline company after it lost his luggage. Sadly, he lost his case.
    33.44  45.53  I was wondering why the ball was getting bigger. Then it hit me.
    47.93  31.58  Never trust an atom, they make up everything!
     0.00   9.00  Waking up this morning was an eye-opening experience
     0.00  22.30  Getting the ability to fly would be so uplifting.
    16.40  37.52  England doesn't have a kidney bank, but it does have a Liverpool.
    33.90  38.07  

In [25]:
neuronpedia_features = dict()

In [52]:
from collections import Counter

def count_features(t):
    fc = Counter()

    for i in range(t.size(0)):
        top_indices = torch.topk(t[i], k=100).indices
        # print(f"\n\n{pun}")
        for idx in top_indices:
            idx_val = idx.item()
            val = t[i, idx_val].item()
    
            if val < 0.1:
                break
    
            fc[idx_val] += 1

    return fc

pun_tensor = torch.stack([f[1] for f in pun_acts])
non_pun_tensor = torch.stack([f[1] for f in non_pun_acts])

def print_out_pun_feats(c):
    for (index, count) in c.most_common(50):
        idx_val = index
        feature_info = neuronpedia_features.get(idx_val)
        if not feature_info:
            feature_info = get_neuronpedia_feature(idx_val)
            neuronpedia_features[idx_val] = feature_info
    
        description = get_feature_description(feature_info)
        
        print(f"\n\nFeature {idx_val:<10} {count} {description}")
        for j, pun in enumerate(puns):
            print(f"{pun_tensor[j, idx_val].item():.6f} {pun}")
    
        print()
        for j, non_pun in enumerate(non_puns):
            print(f"{non_pun_tensor[j, idx_val].item():.6f} {non_pun}")


fc = count_features(pun_tensor)
nfc = count_features(non_pun_tensor)

print_out_pun_feats(fc)




Feature 11530      20 expressions related to analysis and evaluation of methods or interventions
69.143250 To the guy who invented zero, thanks for nothing.
55.242065 What did the grape say when it got crushed? Nothing, it just let out a little wine.
61.104370 Time flies like an arrow. Fruit flies like a banana.
74.663574 Ladies, if he can’t appreciate your fruit jokes, you need to let that mango.
67.091003 A man sued an airline company after it lost his luggage. Sadly, he lost his case.
53.383026 I was wondering why the ball was getting bigger. Then it hit me.
61.300842 Never trust an atom, they make up everything!
60.552063 Waking up this morning was an eye-opening experience
70.825470 Getting the ability to fly would be so uplifting.
79.012054 England doesn't have a kidney bank, but it does have a Liverpool.
68.677368 German sausage jokes are just the wurst.
77.950104 I bought a boat because it was for sail.
62.271973 A cross-eyed teacher couldn’t control his pupils.
72.322266 The

In [53]:
print_out_pun_feats(nfc)



Feature 11530      20 expressions related to analysis and evaluation of methods or interventions
69.143250 To the guy who invented zero, thanks for nothing.
55.242065 What did the grape say when it got crushed? Nothing, it just let out a little wine.
61.104370 Time flies like an arrow. Fruit flies like a banana.
74.663574 Ladies, if he can’t appreciate your fruit jokes, you need to let that mango.
67.091003 A man sued an airline company after it lost his luggage. Sadly, he lost his case.
53.383026 I was wondering why the ball was getting bigger. Then it hit me.
61.300842 Never trust an atom, they make up everything!
60.552063 Waking up this morning was an eye-opening experience
70.825470 Getting the ability to fly would be so uplifting.
79.012054 England doesn't have a kidney bank, but it does have a Liverpool.
68.677368 German sausage jokes are just the wurst.
77.950104 I bought a boat because it was for sail.
62.271973 A cross-eyed teacher couldn’t control his pupils.
72.322266 The

In [57]:
delta_fc = fc - nfc

print_out_pun_feats(delta_fc)



Feature 109859     15 humorous elements and wordplay in the text
18.620703 To the guy who invented zero, thanks for nothing.
28.547129 What did the grape say when it got crushed? Nothing, it just let out a little wine.
11.512590 Time flies like an arrow. Fruit flies like a banana.
51.748695 Ladies, if he can’t appreciate your fruit jokes, you need to let that mango.
0.000000 A man sued an airline company after it lost his luggage. Sadly, he lost his case.
46.442047 I was wondering why the ball was getting bigger. Then it hit me.
26.576075 Never trust an atom, they make up everything!
55.959633 Waking up this morning was an eye-opening experience
77.437454 Getting the ability to fly would be so uplifting.
47.439430 England doesn't have a kidney bank, but it does have a Liverpool.
55.697113 German sausage jokes are just the wurst.
51.950287 I bought a boat because it was for sail.
53.048294 A cross-eyed teacher couldn’t control his pupils.
62.292587 The machine at the coin factory just

In [29]:
delta_nfc = nfc - fc
print_out_pun_feats(delta_nfc)



Feature 58166      8 punctuation marks and formatting indicators commonly used in writing
0.000000 To the guy who invented zero, thanks for nothing.
0.000000 What did the grape say when it got crushed? Nothing, it just let out a little wine.
0.000000 Time flies like an arrow. Fruit flies like a banana.
0.000000 Ladies, if he can’t appreciate your fruit jokes, you need to let that mango.
0.000000 A man sued an airline company after it lost his luggage. Sadly, he lost his case.
0.000000 I was wondering why the ball was getting bigger. Then it hit me.
0.000000 Never trust an atom, they make up everything!
0.000000 Waking up this morning was an eye-opening experience
15.144510 Getting the ability to fly would be so uplifting.
0.000000 England doesn't have a kidney bank, but it does have a Liverpool.
0.000000 German sausage jokes are just the wurst.
11.709650 I bought a boat because it was for sail.
11.179594 A cross-eyed teacher couldn’t control his pupils.
0.000000 The machine at the co

In [30]:
oneliners = [
    "The adjective for metal is metallic, but not so for iron, which is ironic.",
    "Together, I can beat schizophrenia.",
    "“DO NOT TOUCH” must be one of the most terrifying things to read in braille.",
    "My wife asked me to put ketchup on the shopping list that I was making and now, I can’t read anything.",
    "If there's one thing that makes me throw up, it's a dart board on a ceiling.",
    "I don’t like people who take drugs, for example: airport security.",
    "You can't spell advertisements without semen between the tits.",
    "I was gonna tell a time travelling joke but you guys didn't like it",
    "My wife threatened to divorce me when I said I was going to give our daughter a silly name, so I called her Bluff.",
    "man addicted to drinking brake fluid claims he can stop anytime he wants",
    "Gravity is one of the most fundamental forces in the universe, but if you remove it, you get gravy."
]

oneliner_acts = []
for oneliner in oneliners:
    oneliner_acts.append((get_last_token_features(oneliner), get_last_token_features(oneliner + 3 * " ")))


for idx in sorted(joke_features.keys()):
    description = joke_features[idx]
    
    # Print stats with description
    print(f"{idx:<10}  {description}")
    
    if not all(f[1][idx] == 0.0 and f[0][idx] == 0.0 for f in oneliner_acts):
        for j, oneliner in enumerate(oneliners):
            print(f"   {oneliner_acts[j][0][idx].item():6.2f} {oneliner_acts[j][1][idx].item():6.2f}  {oneliner}")



1393        expressions of personal feelings or states of being
2221        expressions of personal sentiment and emotions
2730        references to humor or jokes
     0.00   9.04  The adjective for metal is metallic, but not so for iron, which is ironic.
     0.00   0.00  Together, I can beat schizophrenia.
     0.00   0.00  “DO NOT TOUCH” must be one of the most terrifying things to read in braille.
     0.00  32.07  My wife asked me to put ketchup on the shopping list that I was making and now, I can’t read anything.
     0.00   0.00  If there's one thing that makes me throw up, it's a dart board on a ceiling.
     0.00  10.04  I don’t like people who take drugs, for example: airport security.
    13.05   7.76  You can't spell advertisements without semen between the tits.
    27.60  15.05  I was gonna tell a time travelling joke but you guys didn't like it
     0.00  68.38  My wife threatened to divorce me when I said I was going to give our daughter a silly name, so I called her 

In [32]:
oneliner_tensor = torch.stack([f[0] for f in oneliner_acts])
oneliners_fc = count_features(oneliner_tensor)
doubled_oneliners_fc = Counter({k: v*2 for k, v in oneliners_fc.items()})
oneliners_fc_delta = doubled_oneliners_fc - fc


def print_out_oneliner_feats(c):
    for (index, count) in c.most_common(50):
        idx_val = index
        feature_info = neuronpedia_features.get(idx_val)
        if not feature_info:
            feature_info = get_neuronpedia_feature(idx_val)
            neuronpedia_features[idx_val] = feature_info
    
        description = get_feature_description(feature_info)
        print(f"\n\nFeature {idx_val:<10} {count} {description}")

        print()
        for j, oneliner in enumerate(oneliners):
            print(f"{oneliner_tensor[j, idx_val].item():.6f} {oneliner}")

        print("--- puns --- ")
        for j, pun in enumerate(puns):
            print(f"{pun_tensor[j, idx_val].item():.6f} {pun}")
    

print_out_oneliner_feats(oneliners_fc_delta)



Feature 22571      12 terms related to fitness center management and staffing considerations

25.368475 The adjective for metal is metallic, but not so for iron, which is ironic.
24.325378 Together, I can beat schizophrenia.
17.465916 “DO NOT TOUCH” must be one of the most terrifying things to read in braille.
17.929987 My wife asked me to put ketchup on the shopping list that I was making and now, I can’t read anything.
12.591631 If there's one thing that makes me throw up, it's a dart board on a ceiling.
27.624680 I don’t like people who take drugs, for example: airport security.
23.650333 You can't spell advertisements without semen between the tits.
0.000000 I was gonna tell a time travelling joke but you guys didn't like it
16.559099 My wife threatened to divorce me when I said I was going to give our daughter a silly name, so I called her Bluff.
39.132309 man addicted to drinking brake fluid claims he can stop anytime he wants
8.035021 Gravity is one of the most fundamental for

In [34]:
fc_delta_oneliners = fc - doubled_oneliners_fc
print_out_oneliner_feats(fc_delta_oneliners)



Feature 40174      11 phrases expressing uncertainty or conflict in situations

14.228338 The adjective for metal is metallic, but not so for iron, which is ironic.
0.000000 Together, I can beat schizophrenia.
0.000000 “DO NOT TOUCH” must be one of the most terrifying things to read in braille.
0.000000 My wife asked me to put ketchup on the shopping list that I was making and now, I can’t read anything.
0.000000 If there's one thing that makes me throw up, it's a dart board on a ceiling.
0.000000 I don’t like people who take drugs, for example: airport security.
0.000000 You can't spell advertisements without semen between the tits.
0.000000 I was gonna tell a time travelling joke but you guys didn't like it
0.000000 My wife threatened to divorce me when I said I was going to give our daughter a silly name, so I called her Bluff.
0.000000 man addicted to drinking brake fluid claims he can stop anytime he wants
7.842202 Gravity is one of the most fundamental forces in the universe, b

In [55]:
total_fires = 0

for idx in stricter_filtered_joke_features(joke_features):
    idx_val = idx
    
    if not (pun_tensor[:, idx_val] == 0).all().item() or not (oneliner_tensor[:, idx_val] == 0).all().item() or not (diff_tensor[:, idx_val] == 0).all().item():
        total_fires += 1

print(total_fires, len(stricter_filtered_joke_features(joke_features)))

16 49
